In [ ]:
!pip install sentence_transformers

# Importing Packages

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from nltk.stem import WordNetLemmatizer
import nltk
from sentence_transformers import *
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
import matplotlib.cm as cm
import torch
from nlp_package_pv import *
import imagehash
import cv2
import keras
import tqdm
from tqdm.auto import tqdm as tqdmp
tqdmp.pandas()

# Importing Data

In [ ]:
train_path='../input/shopee-product-matching/train_images/'
train=pd.read_csv('../input/shopee-product-matching/train.csv')
test=pd.read_csv('../input/shopee-product-matching/test.csv')

In [ ]:
train.head()

In [ ]:
# Using the preprocessing function to preprocess the tweet data
preprocess_tweet_data(train,'title')
# Using tokenizer and removing the stopwords
rem_stopwords_tokenize(train,'title')
# Converting all the texts back to sentences
make_sentences(train,'title')

In [ ]:
train.head()

# Importing Model From Sentence Transformer

In [ ]:
model=torch.load('../input/bert-model-gpu-sentence-transformer/Large_bert_model.h5')
# device = torch.device("cuda")
# model.to(device)

In [ ]:
# Getting a model
# model=SentenceTransformer('bert-large-nli-mean-tokens')

# Converting The Text To Word Embeddings

In [ ]:
embeddings = model.encode(train['title'])

# Getting Relatable Text From The Data And It's Score

In [ ]:
query='paper bag victoria secret'
print(query)

In [ ]:
query_embedding = model.encode(query)
top_k=5
cos_scores = util.pytorch_cos_sim(query_embedding, embeddings)[0]
cos_scores = cos_scores.cpu()

#We use torch.topk to find the highest 5 scores
top_results = torch.topk(cos_scores, k=top_k)

print("\n\n======================\n\n")
print("Item Text:", query)
print("\nTop 5 most similar sentences in corpus:")
image_id=[]
for score, idx in zip(top_results[0], top_results[1]):
    image_id.append(train['image'].values[idx])
    print(train['title'].values[idx], "(Score: %.4f)" % (score),"Image Id :%s" % train['image'].values[idx])

# Showing Similar Data

In [ ]:
# Making a function to convert the image to fixed length for subplots
def show_image(main_path,image_path,dim=(256,256)):
    im=cv2.imread(main_path+image_path)
    im=cv2.cvtColor(im,cv2.COLOR_BGR2RGB)
    im=cv2.resize(im,dim)
    plt.imshow(im)


In [ ]:
# Let's have a look at the similar images
for i in range(2):
    plt.subplot(1,2,i+1)
    show_image(train_path,image_id[i])


# Let's try to improve this method using phash

## Let's Try to make a dataframe which has top 50 results using above method

In [ ]:
def preprocess_text(dataset,name):
    # Using the preprocessing function to preprocess the tweet data
    preprocess_tweet_data(dataset,name)
    # Using tokenizer and removing the stopwords
    rem_stopwords_tokenize(dataset,name)
    # Converting all the texts back to sentences
    make_sentences(dataset,name)
def make_embeddings(dataset,name):
    embeddings = model.encode(dataset[name])
    return embeddings
    

# Making a function to return results

In [ ]:
# We Define a threshold for our score so we get reasonable results
def return_results(dataset=train,embedding=embeddings,number=50,ultimate_point=0.95):
    result=[]
    if number>len(dataset):
        number=len(dataset)
    for i in dataset['title'].values :
        query_embedding = model.encode(i)
        top_k=number
        cos_scores = util.pytorch_cos_sim(query_embedding, embedding)[0]
        cos_scores = cos_scores.cpu()

        #We use torch.topk to find the highest 5 scores
        top_results = torch.topk(cos_scores, k=top_k)
        s=''
        count=0
        for score, idx in zip(top_results[0], top_results[1]):
            if score>=ultimate_point :
                count+=1
                if count==1:
                    s+=dataset['posting_id'].values[idx]
                else:
                    s+=' '+dataset['posting_id'].values[idx]
        result.append(s)
    return result
        
                
        
        

In [ ]:
# Let's try this on the test_data
# Preprocessing text on test data
preprocess_text(test,'title')

# Creating Embeddings for the test data title columns
embeddings_test=make_embeddings(test,'title')



In [ ]:
return_results(test,embeddings_test)

# Adding result to submission file

In [ ]:
ss=pd.read_csv('../input/shopee-product-matching/sample_submission.csv')

In [ ]:
ss.head()

In [ ]:
ss['matches']=return_results(test,embeddings_test)

In [ ]:
ss.to_csv('submission.csv',index=False)

In [ ]:
ss.head()

# Up till now we predicted similar products using just text :)